In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from numpy import random
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader
from collections import deque
import numpy as np
import multiprocessing
import time

torch.manual_seed(1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
#rotate a list by given offset, (negative offset, right to left) 
def rotateList(items, offset):
    return np.roll(items,offset)
    
def gaussiantimeseries(limit = 60, interval = 2):
    data = {}
    for t in range(0,limit,interval):
        data[t]=random.normal(loc = random.uniform(1,100), scale =1.5, size =1000)
    return data

def createsignal(limit = 60, interval = 2):
    signal1, signal2, offset = [],[],[]
    offset = random.randint(-5,5)
    data = gaussiantimeseries(limit, interval)
    for t in data:
        dist = data[t]
        signal1.append(random.choice(dist))
        signal2.append(random.choice(dist))
    return signal1, rotateList(signal2,offset), offset

def createsignals(num = 50, limit = 60, interval = 2):
    input_seq = []
    for i in range(num):
        sig1, sig2, offset = createsignal(limit = limit)
        train_data = np.stack((sig1,sig2)).astype(float)
        # scaler = MinMaxScaler(feature_range=(-1, 1))
        # train_data_normalized = scaler.fit_transform(train_data.reshape(-1, 1))
        # train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1)
        # train_label = torch.FloatTensor((offset))
        train_data = torch.FloatTensor(train_data.reshape(-1,1)).view(-1)
        input_seq.append((train_data, torch.FloatTensor([offset])))
    return input_seq
# print(createsignal())

Creating Data

In [ ]:
train_input = createsignals(num=1000)
test_input = createsignals()
# training_length = int(len(input)*(.8))
# train_input = input[:training_length]
# val_input = input[training_length:] 

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [ ]:
model = LSTM()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
epochs = 150
model.train()
for epoch in range(epochs):
    print("Epoch {}/{}".format(epoch+1, epochs))
    print('-' * 10)
    epoch_loss = 0.0
    running_loss = 0.0
    model.zero_grad()
    for seq, labels in train_input:

        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))
        # seq, labels = seq.to(device), labels.to(device)
        y_pred = model(seq)#.to(device)
        optimizer.zero_grad()
        loss = criterion(y_pred, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += y_pred.shape[0] * loss.item()

        # print statistics
        # running_loss += loss.item()
        # if i % 25 == 1:    
        #     print('[%d] loss: %.3f' %
        #           (i + 1, running_loss / 25))
        #     running_loss = 0.0
    print("Epoch {} Loss {}".format(epoch+1, epoch_loss / len(train_input)))

    # if i%25 == 1:
#     print(f'epoch: {i:3} loss: {loss.item():10.8f}')

# print(f'epoch: {i:3} loss: {loss.item():10.10f}')

Epoch 1/150
----------
Epoch 1 Loss 8.312375104427337
Epoch 2/150
----------
Epoch 2 Loss 8.292338802918792
Epoch 3/150
----------
Epoch 3 Loss 8.282923031575978
Epoch 4/150
----------
Epoch 4 Loss 8.275809908423573
Epoch 5/150
----------
Epoch 5 Loss 8.269733278183267
Epoch 6/150
----------
Epoch 6 Loss 8.264219620574732
Epoch 7/150
----------
Epoch 7 Loss 8.259067616251821
Epoch 8/150
----------
Epoch 8 Loss 8.254198277965187
Epoch 9/150
----------
Epoch 9 Loss 8.249534372512251
Epoch 10/150
----------
Epoch 10 Loss 8.245012200379744
Epoch 11/150
----------
Epoch 11 Loss 8.240585761721245
Epoch 12/150
----------
Epoch 12 Loss 8.236216345920493
Epoch 13/150
----------
Epoch 13 Loss 8.231864193550079
Epoch 14/150
----------
Epoch 14 Loss 8.227485812020022
Epoch 15/150
----------
Epoch 15 Loss 8.223024243852123
Epoch 16/150
----------
Epoch 16 Loss 8.218405346930957
Epoch 17/150
----------
Epoch 17 Loss 8.213542964185589
Epoch 18/150
----------
Epoch 18 Loss 8.208328154950403
Epoch 19/1

In [ ]:
epochs = 1
model.eval()
for epoch in range(epochs):
    print("Epoch {}/{}".format(epoch+1, epochs))
    print('-' * 10)
    epoch_loss = 0.0
    running_loss = 0.0
    # model.zero_grad()
    for seq, labels in test_input:

        # model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
        #                 torch.zeros(1, 1, model.hidden_layer_size))
        # # seq, labels = seq.to(device), labels.to(device)
        y_pred = model(seq)#.to(device)
        print(f"Offset {labels.data[0]}, Prediction {y_pred.data[0]}")
        # optimizer.zero_grad()
        loss = criterion(y_pred, labels)
        # loss.backward()
        # optimizer.step()
        epoch_loss += y_pred.shape[0] * loss.item()

        # print statistics
        # running_loss += loss.item()
        # if i % 25 == 1:    
        #     print('[%d] loss: %.3f' %
        #           (i + 1, running_loss / 25))
        #     running_loss = 0.0
    print("Epoch {} Loss {}".format(epoch+1, epoch_loss / len(train_input)))

    # if i%25 == 1:
#     pr

Epoch 1/1
----------
Offset 1.0, Prediction 3.4554266929626465
Offset 2.0, Prediction 3.525254249572754
Offset 0.0, Prediction 1.7010257244110107
Offset -1.0, Prediction -1.0065683126449585
Offset 1.0, Prediction -2.01401948928833
Offset 4.0, Prediction 3.249332904815674
Offset 0.0, Prediction 3.0753164291381836
Offset 4.0, Prediction 2.066652774810791
Offset 1.0, Prediction 1.8922733068466187
Offset 0.0, Prediction 2.023402214050293
Offset 2.0, Prediction 2.4332070350646973
Offset 1.0, Prediction -3.5639262199401855
Offset -1.0, Prediction -4.69520378112793
Offset -4.0, Prediction -1.0466980934143066
Offset -4.0, Prediction -3.4813098907470703
Offset -2.0, Prediction -4.020549774169922
Offset -1.0, Prediction -1.4216910600662231
Offset -2.0, Prediction 3.278092861175537
Offset 2.0, Prediction 1.894456386566162
Offset 4.0, Prediction 3.233872413635254
Offset -2.0, Prediction 1.8103532791137695
Offset -4.0, Prediction -0.5177187323570251
Offset -3.0, Prediction -1.273093581199646
Offset